<a id='Topic0'></a>
# Analyzing the Impact of Climate Change on U.S. Crop Yields

In this notebook, explore the relationship between climate change and crop yields in the United States. More specifically, in this notebook, we will:

1.  Extract, clean, and preprocess crop yield, production, and area planted data sourced from the agricultural survey data conducted by the National Agricultural Statistics Service (NASS), a division of the United States Department of Agriculture (USDA).

2. Extract, clean, and preprocess climate data sourced from weather stations operated by the National Oceanic and Atmospheric Administration (NOAA). These weather stations capture daily records of several climate variables. For this analysis, we are only interested in maximum temperature, minimum temperature, average temperature, and precipitation.

3. Merge the two datasets and visualize the data to uncover potential trends or insights that can inform our understanding of how temperature variations impact agricultural productivity. 

This analysis is particularly valuable for making data-driven decisions in agricultural planning and forecasting.

<a href='#toc'>TOC</a>

In [2]:
# Import libraries
import pandas as pd
import urllib.parse
import altair as alt
from vega_datasets import data

### Extract, clean, and Preprocess Crop Yield Data from the USDA Database

The crop yield, production, and area planted data will be retrieved from the National Agricultural Statistics Service (NASS) QuickStats database. This database will be programmatically accessed using the QuickStats API Key. For this analysis, we will focus on retrieving the yield data and metadata for the top 4 crops grown in the US. We will further narrow our scope to include only the states that are the largest producers of the crops of interest:

1. Corn grown in Iowa
2. Soybean grown in Illinois
3. Barley grown in Idaho
4. Oats grown in South Dakota

In [3]:
# Use the QuickStats API key to access the database and retrieve and return the crop yield data
def get_data(parameters):

    base_url = 'http://quickstats.nass.usda.gov/api/api_GET/?key=D48CBA42-D6F6-31A1-9EF1-46D1A70988F5&' # Base URL which contains the QuickStats API Key
    full_url = base_url + parameters
            
    df = pd.read_csv(full_url)
            
    return df  

In [4]:
# Define the filter_list variable according to our crop selection criteria that we will use for our analysis
filter_list = [
    ('IOWA','CORN','PRODUCTION','CORN, GRAIN - PRODUCTION, MEASURED IN BU'),
    ('IOWA','CORN','YIELD','CORN, GRAIN - YIELD, MEASURED IN BU / ACRE'),
    ('IOWA','CORN','AREA PLANTED','CORN - ACRES PLANTED'),
    ('ILLINOIS','SOYBEANS','PRODUCTION','SOYBEANS - PRODUCTION, MEASURED IN BU'),
    ('ILLINOIS','SOYBEANS','YIELD','SOYBEANS - YIELD, MEASURED IN BU / ACRE'),
    ('ILLINOIS','SOYBEANS','AREA PLANTED','SOYBEANS - ACRES PLANTED'),
    ('IDAHO','BARLEY','PRODUCTION','BARLEY - PRODUCTION, MEASURED IN BU'),
    ('IDAHO','BARLEY','YIELD','BARLEY - YIELD, MEASURED IN BU / ACRE'),
    ('IDAHO','BARLEY','AREA PLANTED','BARLEY - ACRES PLANTED'),
    ('SOUTH DAKOTA','OATS','PRODUCTION','OATS - PRODUCTION, MEASURED IN BU'),
    ('SOUTH DAKOTA','OATS','YIELD','OATS - YIELD, MEASURED IN BU / ACRE'),
    ('SOUTH DAKOTA','OATS','AREA PLANTED','OATS - ACRES PLANTED')
    ]

crop_yield_df = pd.DataFrame()


# Iterate through the filter_list to create the final crop_df dataframe that will be used for our analysis
for state,commodity,stat_desc,description in filter_list:
    source_desc = 'source_desc=SURVEY'
    sector_desc = '&sector_desc=CROPS'
    commodity_desc = f'&commodity_desc={commodity}'
    statisticcat_desc = '&statisticcat_desc=' + urllib.parse.quote(stat_desc)
    short_desc = '&short_desc=' + urllib.parse.quote(description)
    location_desc = '&location_desc='+ urllib.parse.quote(state)
    year__GE = '&year__GE=2013'
    format = '&format=CSV'

    parameters = source_desc + sector_desc + commodity_desc + statisticcat_desc + short_desc + location_desc + year__GE + format

    df = get_data(parameters) # Retrieve the crop yield data based on criteria defined in the filter list element
    df = df[(df['year']!=2024)&(df['reference_period_desc']=='YEAR')] # Filter the dataframe to only include the full year data, and filter out 2024 data 
    df = df[['commodity_desc','statisticcat_desc','year','location_desc','Value']] # Reduce the dataframe size to only include our features of interest
    df.rename(columns={'commodity_desc': 'CROP', 'statisticcat_desc': 'STAT_CAT','year':'YEAR','location_desc':'STATE','Value':'VALUE'}, inplace=True) # Rename fields 

    # Convert string values for Yield, Area Planted and Production into int
    if (df['VALUE'].dtype == 'object'):
        df['VALUE']=df['VALUE'].str.replace(',', '', regex=False)
    df['VALUE'] = df['VALUE'].astype(int)
    df['YEAR'] = df['YEAR'].astype(str)

    # Concatenate the dataframe to the final crop_df which will contain yield data for all 4 crops
    crop_yield_df = pd.concat([crop_yield_df, df], ignore_index=True)

# Replace state names with state abbreviations in order to merge with the climate data
state_name = ['IOWA','ILLINOIS','IDAHO','SOUTH DAKOTA']
state_abb = ['IA','IL','ID','SD']
crop_yield_df['STATE'] = crop_yield_df['STATE'].replace(state_name, state_abb)

# Print top 5 records on the final Crop Yield dataframe
crop_yield_df.head() 


,CROP,STAT_CAT,YEAR,STATE,VALUE
0,CORN,PRODUCTION,2023,IA,2522550000
1,CORN,PRODUCTION,2022,IA,2470000000
2,CORN,PRODUCTION,2021,IA,2539800000
3,CORN,PRODUCTION,2020,IA,2283300000
4,CORN,PRODUCTION,2019,IA,2564100000


### Extract, clean, and Preprocess Climate Data from the NOAA Database

 The climate data is sourced from weather stations operated by the National Oceanic and Atmospheric Administration (NOAA). These weather stations capture daily records of several climate variables. For this analysis, we will retrieve maximum monthly temperature, minimum monthly temperature, average monthly temperature, and monthly precipitation (mm). The weather station data was exported from the NOAA website as CSV files. Weather stations were selected based on proximity to growing regions, and will be used for our analysis to approximate climate during growing season. Please refer to the "Secondary Dataset" page of the accompanying document for additional weather station location details. For our analysis, we will use climate data only for the growing months of the crops of interest:

 1. Growing months for Corn: June - August
 2. Growing months for Soybean: June - August
 3. Growing months for Barley: May - June
 4. Growing months for Oats: May - June


In [5]:
# Retrieve the climate data from csv files exported from the NOAA Database
def read_climate_data(path, crop, state):
    dfs = []
    
    df = pd.read_csv(path)
    # Select columns
    df = df[["NAME", "DATE", "PRCP", "TAVG", "TMAX", "TMIN"]]

    # Extract State Abbreviation and filter df for state of interest
    df["STATE"] = df["NAME"].str.extract(r",\s*([A-Z]{2})\s")[0]
    df = df.drop(columns=["NAME"])
    df = df[df["STATE"]==state]

    # Extract Year and Month
    df["YEAR"] = df["DATE"].str.split("-").str[0]
    df["MONTH"] = df["DATE"].str.split("-").str[1]
    df = df.drop(columns={"DATE"})

    # Map of numerical month value to month
    month_mapping = {
        "01": "January",
        "02": "February",
        "03": "March",
        "04": "April",
        "05": "May",
        "06": "June",
        "07": "July",
        "08": "August",
        "09": "September",
        "10": "October",
        "11": "November",
        "12": "December",
    }

    # Replace numerical month value with month
    df["MONTH"] = df["MONTH"].replace(month_mapping)

    # Create column for corresponding crop type
    df["CROP"] = crop
    dfs.append(df)

    #Combining dataframes of the same crop
    df_combined = pd.concat(dfs, ignore_index = True)

    return df_combined


In [6]:
# Group by year and state and aggregate the data to return the mean TAVG, mean TMIN, mean TMAX, and mean PRCP
def aggregate_data(temp_df,growing_months):
    temp_df = temp_df[temp_df['MONTH'].isin(growing_months)].dropna()
    temp_df = temp_df.groupby(['YEAR','STATE']).agg({'PRCP': 'mean', 'TMIN': 'mean', 'TMAX': 'mean', 'TAVG': 'mean'}).reset_index()

    return temp_df


In [7]:
# Relative paths for .csv data for each crop
corn_paths = "noaa_csv_files/Corn_1.csv"
soybean_paths = "noaa_csv_files/Soybean_1.csv"
barley_paths = "noaa_csv_files/Barley_1.csv"
oats_paths = "noaa_csv_files/Oats_1.csv"

# List of crop, state they are grown, and growing months for the crop 
crops = [
    ("corn","IA",['June', 'July', 'August']), 
    ("soybean","IL",['June', 'July', 'August']), 
    ("barley","ID",['May', 'June']), 
    ("oats","SD",['May', 'June'])
    ]

#List of .csv paths
paths = [corn_paths, soybean_paths, barley_paths, oats_paths]

#Use for loop and read_cimate_data() fo create climate dataframes for each state and crop and save it to the temp_data dictionary
climate_data = {}
for path, (crop,state,growing_months) in zip(paths, crops):
    climate_df = read_climate_data(path, crop, state)
    climate_data[crop] = aggregate_data(climate_df, growing_months)

# Print top 5 records of the final climate dataframe for corn
climate_data["corn"].head() 



,YEAR,STATE,PRCP,TMIN,TMAX,TAVG
0,2013,IA,2.250000,59.566667,79.833333,69.700000
1,2014,IA,5.306667,59.666667,77.633333,68.666667
2,2015,IA,6.083333,59.266667,77.666667,68.466667
3,2016,IA,6.983333,61.200000,80.600000,70.933333
4,2017,IA,2.820000,60.450000,82.500000,71.450000


### Merge the Crop Yield Data with the Climate Data

Now we will combine the crop yield dataframe with the climate dataframes by YEAR and STATE to create our final master dataframe. This dataframe will be used for our exploratory data analysis and visualization. 

In [8]:
# For every crop climate dataframe stored in the climate_data dictionary, merge it with the crop yield data and append it to our final dataframe that will be used for our analysis
final_dfs = []

for climate_df in climate_data.values():
    merge_df = pd.merge(crop_yield_df, climate_df, left_on=['STATE', 'YEAR'], right_on=['STATE', 'YEAR'])
    final_dfs.append(merge_df)

final_df = pd.concat(final_dfs, ignore_index=True)

In [9]:
# Print top 5 records of the final dataframe
final_df.head()

,CROP,STAT_CAT,YEAR,STATE,VALUE,PRCP,TMIN,TMAX,TAVG
0,CORN,PRODUCTION,2023,IA,2522550000,2.676000,59.880000,82.180000,71.040000
1,CORN,PRODUCTION,2022,IA,2470000000,3.153333,61.383333,83.233333,72.333333
2,CORN,PRODUCTION,2021,IA,2539800000,3.386667,61.450000,84.316667,72.866667
3,CORN,PRODUCTION,2020,IA,2283300000,1.855000,62.283333,84.416667,73.350000
4,CORN,PRODUCTION,2019,IA,2564100000,3.758333,60.416667,79.866667,70.133333


### Analysis: Descriptive Statistics for crop yield and climate data

We will first describe the key yield and climate statistics for each crop. This includes mean, standard deviation, min, and max. This can be used to better understand the variability in yield year-over-year for each crop, as well as the unique climate conditions each crop is exposed to.

In [10]:
# Loop through each crop in crop_data
for crop in final_df['CROP'].unique():
    print(f"Descriptive Statistics for {crop}:")

    # Filter final_df for crop and yield
    crop_df = final_df[(final_df['CROP']==crop)&(final_df['STAT_CAT']=='YIELD')]
    crop_df = crop_df.copy() # Create a copy of the DataFrame slice before renaming to avoid warnings
    crop_df.rename(columns={'VALUE': 'YIELD'}, inplace=True)
    crop_df = crop_df[['YIELD','PRCP', 'TAVG', 'TMAX', 'TMIN']]

    # Print descriptive statistics
    print(crop_df.describe().round(2))
    print('')


Descriptive Statistics for CORN:
        YIELD   PRCP   TAVG   TMAX   TMIN
count   11.00  11.00  11.00  11.00  11.00
mean   192.27   4.31  70.99  81.26  60.71
std     13.29   2.28   1.62   2.37   1.07
min    164.00   1.86  68.47  77.63  59.27
25%    185.00   2.75  69.92  79.85  59.77
50%    198.00   3.39  71.04  81.62  60.45
75%    201.50   5.70  72.14  82.87  61.42
max    204.00   9.10  73.35  84.42  62.28

Descriptive Statistics for SOYBEANS:
       YIELD   PRCP   TAVG   TMAX   TMIN
count  11.00  11.00  11.00  11.00  11.00
mean   58.82   3.98  74.23  83.90  64.56
std     4.58   0.93   1.09   1.33   1.12
min    50.00   2.76  72.48  81.42  63.23
25%    56.00   3.15  73.49  82.99  63.50
50%    59.00   4.07  74.23  84.43  64.62
75%    63.00   4.57  75.16  84.88  65.53
max    65.00   5.53  75.72  85.22  66.27

Descriptive Statistics for BARLEY:
        YIELD   PRCP   TAVG   TMAX   TMIN
count   11.00  11.00  11.00  11.00  11.00
mean   101.18   1.60  56.63  71.12  42.15
std      8.10   0.76

### Visualization: Plot a heat map for crop yield and climate variables over time for each crop

We will now create a heatmap to highlight the yield and climate trends over time for each crop

In [17]:
linechart_df = final_df[final_df['STAT_CAT']=='YIELD']
linechart_df = linechart_df.copy()
linechart_df.rename(columns={'VALUE': 'YIELD'}, inplace=True)

# List of metrics to loop over
metrics = ['YIELD', 'PRCP', 'TMIN', 'TMAX', 'TAVG']

# Loop over each metric and create a line chart
for metric in metrics:
    line_chart = alt.Chart(linechart_df).mark_line().encode(
    x=alt.X('YEAR'),
    y=alt.Y(metric),
    color=alt.Color('CROP')
    ).properties(
        title=f'{metric} Trends Over Time',
        width=400,
        height=200   
    )

    line_chart.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

### Visualization: Plot crop yield, production, and area planted over time 

For each crop, we will now plot yield, production, and area planted over time. This will help us understand whether changes in yield were due to reduced production or a decrease in the area planted.

In [18]:
crop_chart_list = []
colors = ['green', 'orange', 'maroon']
offset_list = [0, 0, 60]

# For each crop, plot yield, production and area planted over time
for crop in final_df['CROP'].unique():
    cat_chart_list = []  
    count = 0
    
    # for the crop, create a line chart for yield, production, and area planted over time
    for category in final_df['STAT_CAT'].unique():

        crop_data = final_df[(final_df['CROP'] == crop) & (final_df['STAT_CAT'] == category)]

        color = colors[count]
        offset = offset_list[count]
        count += 1

        # Create line chart
        line_chart = alt.Chart(crop_data).mark_line().encode(
            x=alt.X('YEAR'),
            y=alt.Y('VALUE', 
                    title=category, 
                    axis=alt.Axis(offset=offset),
                    scale=alt.Scale(domain=[
                        crop_data['VALUE'].min() - (crop_data['VALUE'].min() * 0.1), # Adjust the scale of the Y-axis 
                        crop_data['VALUE'].max() + (crop_data['VALUE'].min() * 0.1) # Adjust the scale of the Y-axis 
                    ])),
            color=alt.Color('STAT_CAT', scale=alt.Scale(domain=final_df['STAT_CAT'].unique(), range=colors), title='Category') # Add legend
        ).properties(
            title=f'{crop} Farming Trends Over Time',
            width=400,
            height=200   
        )

        # Append to cat_chart_list
        cat_chart_list.append(line_chart)
            
    # Upack the list and combine the line charts for each crop, using multiple y-axis
    crop_combined = alt.layer(*cat_chart_list).resolve_scale(
        y='independent'  
    ).properties(
        width=200,
        height=100
    )
    
    # Append to crop_chart_list 
    crop_chart_list.append(crop_combined)

# Vertically concat the charts for each crop
concat_bar_charts = alt.vconcat(*crop_chart_list)

concat_bar_charts

alt.VConcatChart(...)

### Analysis: Determine the correlation between crop yield and the climate variables 

We will now use the Pandas .corr() function to compute pairwise correlation of the yield data for each crop and the climate variables

In [26]:
# Empty dictionary to be used to store results for each crop
crop_results = {}

crop_correlations = pd.DataFrame()

# Create the correlation matrix for each crop and store results in crop_results
for crop in final_df['CROP'].unique():
    crop_df = final_df[(final_df['CROP']==crop)&(final_df['STAT_CAT'] == 'YIELD')] # Filter final df for crop of interest and yield values
    crop_df = crop_df.copy() # Create a copy of the DataFrame slice before renaming to avoid warnings
    crop_df.rename(columns={'VALUE': 'YIELD'}, inplace=True)
    crop_df.rename(columns={'VALUE':'YIELD'}, inplace=True) # Rename Value field to Yield
    correlation_matrix = crop_df[['PRCP', 'TAVG', 'TMAX', 'TMIN', 'YIELD']].corr(method='spearman') # Compute Spearman correlation coefficient for each variable

    crop_results[crop] = correlation_matrix

# For every crop correlation matrix, extract only the Yield correlation coefficients with the climate variables
for crop, corr_matrix in crop_results.items():
    yield_corr = corr_matrix['YIELD'].drop('YIELD')  # Drop the Yield-Yield correlation coefficients 
    yield_corr.name = crop # Rename series name to include the specific crop 
    
    crop_correlations = pd.concat([crop_correlations, yield_corr], axis=1) # Combine all the yield correlation data into one dataframe

# Transpose the dataframe
crop_correlations = crop_correlations.T
crop_correlations = crop_correlations[['PRCP', 'TMIN', 'TMAX', 'TAVG']]

print(crop_correlations)


              PRCP      TMIN      TMAX      TAVG
CORN      0.236364  0.290909  0.409091  0.336364
SOYBEANS  0.064372  0.482791  0.708093  0.689701
BARLEY    0.472727  0.027273 -0.372727 -0.209091
OATS      0.298870 -0.400026 -0.622146 -0.516151


### Visualization: Plot the relationship between crop yield and the climate variables 

We will now create a scatter plot of yield verus mean precipitation (PRCP), mean minimum temperature (TMIN), mean maximum temperature (TMAX), and mean average temperature (TAVG) for each crop. We will concatenate the resulting charts together. The relationship that we visualize should closely match the correlation analysis we conducted. 

This plot will help us visualize which crops are more sensitive to climate change.

In [27]:
crop_corr_long = crop_correlations.reset_index().melt(id_vars='index')
crop_corr_long.columns = ['Crop', 'Climate Variable', 'Correlation']

# Create the heatmap using Altair
heatmap = alt.Chart(crop_corr_long).mark_rect().encode(
    x=alt.X('Climate Variable', title='Climate Variable'),
    y=alt.Y('Crop:O', title='Crop'),
    color=alt.Color(
        'Correlation:Q', 
        scale=alt.Scale(scheme='redblue', domain=[-1, 1]),  # Diverging color scale from -1 to 1
        legend=alt.Legend(title='Spearman Correlation')
    )
).properties(
    width=400,
    height=400,
    title='Crop Yield vs Climate Variables Correlation Heatmap'
)

# Create text labels for the heatmap
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('Correlation:Q', format='.2f'), 
    color=alt.value('black')
)

# Combine the heatmap with the text labels
heatmap_with_labels = heatmap + text

heatmap_with_labels.show()

alt.LayerChart(...)

### Visualization: Plot the locations of weather stations in each state where the crop of interest is grown.

For each crop, we will plot the location of the weather station used for our climate data. The base map for the United States will be retrieved from the vega_datasets library.

In [14]:
# Create a dataframe that has the city, state and geo location for the weather stations used
city_data = pd.DataFrame({
    'city': ['Ford Dodge', 'Harcourt', 'Clare', 'Badger', 'Decatur', 'Latham', 'Harristown', 'Illiopolis', 
             'Warrensburg', 'Dalton City', 'Oreana', 'Mount Zion', 'Idaho Falls', 'Swan Valley', 
             'Dale Bitner', 'Pine Creek Pass', 'Mobridge', 'Isabel', 'Keldron', 'McIntosh', 'McLaughlin'],
    'state': ['IA', 'IA', 'IA', 'IA', 'IL', 'IL', 'IL', 'IL', 'IL', 'IL', 'IL', 'IL', 'ID', 'ID', 
              'ID', 'ID', 'SD', 'SD', 'SD', 'SD', 'SD'],
    'lat': [42.4975, 42.2628, 42.5889, 42.6144, 39.8403, 39.9687, 39.8539, 39.8537, 39.9328, 
            39.7120, 39.9386, 39.7714, 43.4927, 43.4560, 43.4560, 43.5716, 45.5372, 45.3941, 
            45.9319, 45.9214, 45.8144],
    'lon': [-94.1680, -94.1758, -94.3461, -94.1461, -88.9548, -89.1623, -89.0840, -89.2420, 
            -89.0620, -88.8045, -88.8656, -88.8742, -112.0408, -111.3416, -111.3416, -111.2152, 
            -100.4279, -101.4296, -101.8096, -101.3496, -100.8104],
    'crop': ['corn', 'corn', 'corn', 'corn', 'soybean', 'soybean', 'soybean', 'soybean', 
             'soybean', 'soybean', 'soybean', 'soybean', 'barley', 'barley', 'barley', 
             'barley', 'oats', 'oats', 'oats', 'oats', 'oats']
})

In [15]:
# Create a dataframe that contains the names of the states used for our analysis. This will be overlayed on the map to label the state
us_states_labels = pd.DataFrame({
    'state': ['Idaho', 'Illinois', 'Iowa', 'South Dakota'],
    'lat': [44.1, 40.0, 42.0, 44.0],
    'lon': [-114.3, -89.0, -93.0, -100.0]
})


In [16]:
crop_colors = {
    'corn': 'red',
    'soybean': 'blue',
    'barley': 'green',
    'oats': 'purple'
}

# Create the base map using vega-datasets. For methodology, refer to the following: https://altair-viz.github.io/gallery/us_state_capitals.html
states = alt.topo_feature(data.us_10m.url, feature='states')

us_map = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=700,
    height=400
)

# Encode the state names as a text mark
state_labels = alt.Chart(us_states_labels).mark_text(
    fontSize=12,
    fontWeight='bold',
    dy=-10  
).encode(
    longitude='lon',
    latitude='lat',
    text='state',
    tooltip=['state']
)

# Add to layers list. This list will contain all the charts that will be overlayed for the final visual
layers = [us_map, state_labels]

# For each state that we collected data from, plot the geo location of the weather station
for state in city_data['state'].unique():
    state_df = city_data[city_data['state'] == state]
    
    city_point = alt.Chart(state_df).mark_circle(size=100).encode(
        longitude='lon:Q',
        latitude='lat:Q',
        tooltip=['city'],
        color=alt.Color('crop:N',
                        scale=alt.Scale(domain=list(crop_colors.keys()), 
                                        range=list(crop_colors.values())),
                        legend=alt.Legend(title='Crops'))  
    )

    # Append to layers list
    layers.append(city_point)

# Unpack the layers list and over lay the charts ontop of each other 
highlighted_map = alt.layer(*layers)

highlighted_map

alt.LayerChart(...)